# Параллельная разработка с библиотекой asyncio

## Основы

Здесь сначала введем новый термин - конкурентность (от англ. concurrency). Почти везде слово concurrency на русский переводится как параллелизм, однако это не совсем корректный перевод. Параллелизм - это когда одновременно работают несколько потоков операций. Допустим, у нас есть ПК с 4 ядрами ЦПУ. Тогда параллельно могут работать не более 4 потоков. При этом, если посмотреть на диспетчер задач, то там скорее всего будет числиться гораздо большее число одновременно выполняемых процессов. То есть, операционная система выделяет каждому из потоков этих процессов какой-то квант процессорного времени, чередуя потоки. Это и есть пример конкурентности. 